In [1]:
import sys
import os
import os.path as osp
import pickle
import math
import shutil
import re # For sorting in collect_generated_images
import glob # For finding images in collect_generated_images

import torch
import numpy as np
from PIL import Image as PILImage # Renamed to avoid conflict
import imageio # For GIF creation
from IPython.display import Image as IPImage, display # For displaying GIF

from accelerate import Accelerator
from accelerate.utils import set_seed
from diffusers import StableDiffusionPipeline
from tqdm.auto import tqdm

# --- Add infer.py's directory to sys.path if it's not the current directory ---
# If infer.py is in the same directory as the notebook, this is not strictly needed
# but good practice if you move things around.
# current_notebook_dir = os.getcwd()
# if current_notebook_dir not in sys.path:
#    sys.path.append(current_notebook_dir)

# --- Import components from your infer.py ---
try:
    from infer import Infer, EncoderStatesDataset, collate_fn, TOKEN2ID, UNIQUE_TOKENS
    # If infer.py also has WHICH_MODEL, WHICH_STEP etc. as globals, you could import them too:
    # from infer import WHICH_MODEL as INFER_PY_WHICH_MODEL # alias if needed
except ImportError as e:
    print(f"Error importing from infer.py: {e}")
    print("Please ensure infer.py is in the same directory as the notebook, or in sys.path.")
    print("Also ensure infer.py defines Infer, EncoderStatesDataset, collate_fn, TOKEN2ID, UNIQUE_TOKENS at the top level.")
    raise

# --- Import other necessary custom modules (if not handled by infer.py's imports for the notebook's direct use) ---
# CompassEncoder is needed to instantiate the 'merger' before passing it to the Infer class
try:
    from compass_encoder import CompassEncoder # Or whatever the correct class name is
    from lora_diffusion import inject_trainable_lora # if directly used in notebook for loading
except ImportError as e:
    print(f"Error importing compass_encoder or lora_diffusion: {e}")
    print("Ensure these modules are accessible.")
    raise

# --- Notebook-specific Configurations ---
# You can override or set these independently of infer.py's main block
WHICH_MODEL_NOTEBOOK = "compass" # Or your specific model checkpoint name from infer.py
WHICH_STEP_NOTEBOOK = 20000           # Or your specific checkpoint step from infer.py
KEYWORD_NOTEBOOK = "notebook_interactive"
NUM_INFERENCE_STEPS_NOTEBOOK = 50     # Number of diffusion steps for notebook runs
INFER_BATCH_SIZE_NOTEBOOK = 1         # Batch size for inference (adjust based on GPU memory)
NOTEBOOK_TMP_DIR = f"./tmp_{WHICH_MODEL_NOTEBOOK}_{WHICH_STEP_NOTEBOOK}_{KEYWORD_NOTEBOOK}"
OUTPUT_GIF_FILENAME = f"output_{KEYWORD_NOTEBOOK}.gif"

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU (this will be very slow).")

Using GPU: NVIDIA RTX A6000


In [2]:
def collect_generated_images(tmp_dir, prompt_text_unused, output_gif_path):
    """
    Collects images from the first subdirectory found in tmp_dir (assumed to be the scene's images)
    and creates a GIF.
    """
    image_files = []
    subdirs = [d for d in os.listdir(tmp_dir) if osp.isdir(osp.join(tmp_dir, d))]
    if not subdirs:
        print(f"No subdirectories found in {tmp_dir} to collect images from.")
        return False

    scene_dir_name = subdirs[0]
    scene_dir_path = osp.join(tmp_dir, scene_dir_name)
    print(f"Collecting images for GIF from: {scene_dir_path}")

    for ext in ("*.jpg", "*.png", "*.jpeg"):
        image_files.extend(glob.glob(osp.join(scene_dir_path, ext)))
    
    image_files.sort(key=lambda f: int(re.search(r'(\d+)\.(jpg|png|jpeg)$', f).group(1)) if re.search(r'(\d+)\.(jpg|png|jpeg)$', f) else f)

    if not image_files:
        print(f"No images found in {scene_dir_path}")
        return False

    frames = []
    for image_file in tqdm(image_files, desc="Reading frames for GIF", leave=False):
        try:
            frames.append(imageio.imread(image_file))
        except Exception as e:
            print(f"Skipping {image_file} due to error: {e}")
    
    if frames:
        os.makedirs(osp.dirname(output_gif_path), exist_ok=True)
        imageio.mimsave(output_gif_path, frames, fps=10, loop=0)
        print(f"GIF saved to {output_gif_path}")
        return True
    else:
        print(f"No frames collected, GIF not created.")
        return False

In [3]:
# --- Path to your trained model artifacts ---
CKPT_BASE_PATH = "../ckpts/multiobject/" # Adjust if your ckpt layout is different

args_path = osp.join(CKPT_BASE_PATH, f"__{WHICH_MODEL_NOTEBOOK}", "args.pkl")
training_state_path = osp.join(CKPT_BASE_PATH, f"__{WHICH_MODEL_NOTEBOOK}", f"training_state_{WHICH_STEP_NOTEBOOK}.pth")

if not osp.exists(args_path):
    raise FileNotFoundError(f"args.pkl not found at {args_path}. Please check WHICH_MODEL_NOTEBOOK and paths.")
if not osp.exists(training_state_path):
    raise FileNotFoundError(f"Training state .pth not found at {training_state_path}. Please check WHICH_MODEL_NOTEBOOK, WHICH_STEP_NOTEBOOK and paths.")

with open(args_path, "rb") as f:
    loaded_args_notebook = pickle.load(f) # These are the args from the training run
    print("Loaded args.pkl from training:", loaded_args_notebook)

# Initialize pipeline
pipeline = StableDiffusionPipeline.from_pretrained(
    loaded_args_notebook.get("pretrained_model_name_or_path", "stabilityai/stable-diffusion-2-1"), # Use model from args if specified
    torch_dtype=torch.float32
)
pipeline = pipeline.to(device)

# Initialize merger (Pose Embedding MLP)
merger = CompassEncoder(loaded_args_notebook['merged_emb_dim']) # merged_emb_dim from loaded_args

print(f"Loading training state from: {training_state_path}")
training_state = torch.load(training_state_path, map_location="cpu")

# Load LoRA weights for UNet if trained (using logic from infer.py's main)
if loaded_args_notebook.get('train_unet', False):
    print("Injecting and loading LoRA weights for UNet...")
    # Assuming inject_trainable_lora expects target_replace_module, you might need to pass it if it's in loaded_args_notebook
    # For simplicity, using a common default. Adjust if your lora_diffusion.py has different needs.
    target_modules = loaded_args_notebook.get('lora_target_modules', ["CrossAttention", "Attention", "GEGLU"]) # Example, check your args
    
    # Check if inject_trainable_lora is from the original lora_diffusion or your modified one
    # The original from cloneofsimo might not return unet_lora_params
    # Assuming your version of inject_trainable_lora behaves as expected
    try:
        # If your inject_trainable_lora is the one that modifies the model in-place
        # and you load weights directly using pipeline.unet.load_state_dict
        inject_trainable_lora(pipeline.unet, r=loaded_args_notebook['lora_rank'], target_replace_module=target_modules)
    except TypeError as te:
        print(f"TypeError during inject_trainable_lora: {te}. This might be due to API differences.")
        print("Ensure inject_trainable_lora and its usage match your lora_diffusion.py version.")
        # Fallback or specific handling if needed

    unet_state_dict = pipeline.unet.state_dict()
    pretrained_unet_lora_state_dict = {}
    if "unet" in training_state and "lora" in training_state["unet"]:
         pretrained_unet_lora_state_dict = training_state["unet"]["lora"]
    
    # Load only LoRA parameters by checking keys
    final_unet_state_dict_to_load = {}
    for name, param in pretrained_unet_lora_state_dict.items():
        if name in unet_state_dict : # Make sure the key exists in the current UNet
            final_unet_state_dict_to_load[name] = param
        # if name.find("lora") != -1: # Original check from infer.py
        #    final_unet_state_dict_to_load[name] = param
            
    if final_unet_state_dict_to_load:
        missing, unexpected = pipeline.unet.load_state_dict(final_unet_state_dict_to_load, strict=False)
        print(f"UNet LoRA Load: Missing keys: {len(missing)}, Unexpected keys: {len(unexpected)}")
        if unexpected: print(f"  Unexpected UNet keys: {unexpected[:5]}...") # Print a few if many
    else:
        print("Warning: No LoRA weights found or prepared for UNet in training_state.")
else:
    print("UNet LoRA training was not enabled in args. Skipping UNet LoRA loading.")


# Load merger state
if "merger" in training_state and "model" in training_state["merger"]:
    merger.load_state_dict(training_state["merger"]["model"], strict=False)
    print("Merger (Pose MLP) weights loaded.")
else:
    print("Warning: No merger weights found for 'merger.model' in training_state.")

merger = merger.to(device).eval()
pipeline.unet.eval()
pipeline.vae.eval()
pipeline.text_encoder.eval()

# Initialize Accelerator
accelerator = Accelerator()

# Create Infer instance (imported from infer.py)
# The Infer class from your infer.py should have an __init__ method that matches these arguments.
infer_instance_notebook = Infer(
    merged_emb_dim=loaded_args_notebook['merged_emb_dim'],
    accelerator=accelerator,
    unet=pipeline.unet,
    scheduler=pipeline.scheduler,
    vae=pipeline.vae,
    text_encoder=pipeline.text_encoder,
    tokenizer=pipeline.tokenizer,
    merger=merger,
    tmp_dir=NOTEBOOK_TMP_DIR, # Use notebook-specific temp directory
    bs=INFER_BATCH_SIZE_NOTEBOOK # Use notebook-specific batch size
)
# Pass NUM_INFERENCE_STEPS_NOTEBOOK to the call method if it's used there
# The Infer class provided in the prompt uses a global NUM_INFERENCE_STEPS

print("Infer class (from infer.py) initialized and models loaded.")

Loaded args.pkl from training: {'ada': False, 'wandb': True, 'pretrained_model_name_or_path': 'stabilityai/stable-diffusion-2-1', 'run_name': 'stage1_20000', 'project': 'iisc', 'controlnet_prompts_file': '../prompts/prompts_2410.txt', 'root_data_dir': '', 'pretrained_vae_name_or_path': None, 'revision': None, 'tokenizer_name': None, 'instance_data_dir_1subject': '../training_data_2410/ref_imgs_1subject', 'instance_data_dir_2subjects': '../training_data_2410/ref_imgs_2subjects', 'controlnet_data_dir_1subject': '../training_data_2410/controlnet_imgs_1subject', 'controlnet_data_dir_2subjects': '../training_data_2410/controlnet_imgs_2subjects', 'class_data_dir': '../training_data_2410/prior_imgs', 'include_class_in_prompt': True, 'pose_only_embedding': True, 'normalize_merged_embedding': False, 'use_location_conditioning': False, 'attn_bbox_from_class_mean': True, 'use_ref_images': True, 'use_controlnet_images': True, 'text_encoder_bypass': False, 'appearance_skip_connection': False, 'repl

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading training state from: ../ckpts/multiobject/__compass/training_state_20000.pth
Injecting and loading LoRA weights for UNet...
UNet LoRA Load: Missing keys: 686, Unexpected keys: 0
Merger (Pose MLP) weights loaded.
Infer class (from infer.py) initialized and models loaded.


In [4]:
num_frames_for_gif = 8 # Number of frames for the animation

# --- Define Bounding Boxes ---
bbox_subject1_static = [[0.00, 0.55, 0.45, 1.00]] * (num_frames_for_gif + 1)
bbox_subject2_static = [[0.55, 0.55, 1.00, 1.00]] * (num_frames_for_gif + 1)

# --- Define Scene Data ---
current_scene_subjects_data = [
    {
        "subject": "sedan", # Make sure 'sedan' is in TOKEN2ID (imported from infer.py)
        "normalized_azimuths": np.linspace(0, 1, num_frames_for_gif + 1),
        "bboxes": bbox_subject1_static
    },
    {
        "subject": "suv",   # Make sure 'suv' is in TOKEN2ID (imported from infer.py)
        "normalized_azimuths": 1 - np.linspace(0, 1, num_frames_for_gif + 1),
        "bboxes": bbox_subject2_static
    }
]

prompt_for_scene = "cinematic film still of PLACEHOLDER in the backyard of a bungalow on a sunny afternoon, sharp, high quality, best quality, high resolution, 8k"

print(f"Number of frames to generate: {num_frames_for_gif}")
print(f"Scene setup: {len(current_scene_subjects_data)} subjects.")
# TOKEN2ID and UNIQUE_TOKENS are now from infer.py
print(f"Example TOKEN2ID['sedan']: {TOKEN2ID.get('sedan', 'Not in TOKEN2ID')}")
print(f"Example UNIQUE_TOKENS['0_0']: {UNIQUE_TOKENS.get('0_0', 'Not in UNIQUE_TOKENS')}")

Number of frames to generate: 8
Scene setup: 2 subjects.
Example TOKEN2ID['sedan']: 24237
Example UNIQUE_TOKENS['0_0']: bnha


In [ ]:
if osp.exists(OUTPUT_GIF_FILENAME):
    os.remove(OUTPUT_GIF_FILENAME)
if osp.exists(NOTEBOOK_TMP_DIR): # Clean tmp dir before run
    shutil.rmtree(NOTEBOOK_TMP_DIR)
os.makedirs(NOTEBOOK_TMP_DIR, exist_ok=True)


print(f"Starting inference for prompt: {prompt_for_scene}")
print(f"Output GIF will be saved to: {OUTPUT_GIF_FILENAME}")

# The `__call__` method of the Infer class (from infer.py) will be invoked.
# It needs `args` which are the loaded training arguments.
# Also, NUM_INFERENCE_STEPS is used by the Infer class. If it's a global in infer.py,
# you might need to ensure the notebook's value is used, or modify Infer to accept it.
# For now, assuming Infer class uses a global NUM_INFERENCE_STEPS or it's hardcoded.
# The provided Infer class structure has NUM_INFERENCE_STEPS hardcoded/globally accessible in its methods.
# Let's ensure the 'args' dict passed to __call__ is the one from training.
# And the Infer class from infer.py uses global constants for NUM_INFERENCE_STEPS, etc.
# If your Infer class in infer.py was modified to take NUM_INFERENCE_STEPS in __init__ or __call__, adjust accordingly.

infer_instance_notebook(
    seed=42,
    gif_path=OUTPUT_GIF_FILENAME,
    prompt=prompt_for_scene,
    all_subjects_data=[current_scene_subjects_data],
    args=loaded_args_notebook # Pass the loaded_args from training, as Infer class expects this structure
)

# Display the generated GIF
if osp.exists(OUTPUT_GIF_FILENAME):
    print("Displaying generated GIF:")
    display(IPImage(filename=OUTPUT_GIF_FILENAME))
else:
    print(f"GIF not found at {OUTPUT_GIF_FILENAME}. Check logs in {NOTEBOOK_TMP_DIR} and console output.")

Starting inference for prompt: cinematic film still of PLACEHOLDER in the backyard of a bungalow on a sunny afternoon, sharp, high quality, best quality, high resolution, 8k
Output GIF will be saved to: output_notebook_interactive.gif
every thread finished generating the encoder hidden states...
every thread finished preparing their dataloaders...
starting generation...
0 is doing ['./tmp_compass_20000_notebook_interactive/sedan__suv/000.jpg']


/home/test/anaconda3/envs/contwords/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0 is doing ['./tmp_compass_20000_notebook_interactive/sedan__suv/001.jpg']
0 is doing ['./tmp_compass_20000_notebook_interactive/sedan__suv/002.jpg']
0 is doing ['./tmp_compass_20000_notebook_interactive/sedan__suv/003.jpg']
0 is doing ['./tmp_compass_20000_notebook_interactive/sedan__suv/004.jpg']
0 is doing ['./tmp_compass_20000_notebook_interactive/sedan__suv/005.jpg']
